<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [3]</a>'.</span>

# Input HMP
This notebook pulls the HMP accelerometer sensor data classification data set

In [1]:
import fnmatch
import os
from pathlib import Path
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql.functions import lit
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType
from pyspark.sql.types import StructField
from pyspark.sql.types import StructType
import random
import re
import shutil
import sys

In [2]:
# path and file name for output (default: data.csv)
data_csv = os.environ.get('data_csv', 'data.csv')

# url of master (default: local mode)
master = os.environ.get('master', "local[*]")

# temporal data storage for local execution
data_dir = os.environ.get('data_dir', 'data/')

# sample on input data to increase processing speed 0..1 (default: 1.0)
sample = os.environ.get('sample', '1.0')

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [3]:
# override parameters received from a potential call using %run magic
parameters = list(
    map(
        lambda s: re.sub('$', '"', s),
        map(
            lambda s: s.replace('=', '="'),
            filter(
                lambda s: s.find('=') > -1,
                sys.argv
            )
        )
    )
)

for parameter in parameters:
    exec(parameter)

# cast parameters to appropriate type
sample = float(sample)

Lets create a local spark context (sc) and session (spark)

In [ ]:
sc = SparkContext.getOrCreate(SparkConf().setMaster(master))

spark = SparkSession \
    .builder \
    .getOrCreate()

Lets pull the data in raw format from the source (github)

In [ ]:
!rm -Rf HMP_Dataset
!git clone https://github.com/wchill/HMP_Dataset

In [ ]:
schema = StructType([
    StructField("x", IntegerType(), True),
    StructField("y", IntegerType(), True),
    StructField("z", IntegerType(), True)])

This step takes a while, it parses through all files and folders and creates a temporary dataframe for each file which gets appended to an overall data-frame "df". In addition, a column called "class" is added to allow for straightforward usage in Spark afterwards in a supervised machine learning scenario for example.

In [ ]:
d = 'HMP_Dataset/'

# filter list for all folders containing data (folders that don't start with .)
file_list_filtered = [s for s in os.listdir(d)
                      if os.path.isdir(os.path.join(d, s)) & ~fnmatch.fnmatch(s, '.*')]

# create pandas data frame for all the data

df = None

for category in file_list_filtered:
    data_files = os.listdir('HMP_Dataset/' + category)

    # create a temporary pandas data frame for each data file
    for data_file in data_files:
        if sample < 1.0:
            if random.random() > sample:
                print('Skipping: ' + data_file)
                continue
        print(data_file)
        temp_df = spark.read. \
            option("header", "false"). \
            option("delimiter", " "). \
            csv('HMP_Dataset/' + category + '/' + data_file, schema=schema)

        # create a column called "source" storing the current CSV file
        temp_df = temp_df.withColumn("source", lit(data_file))

        # create a column called "class" storing the current data folder
        temp_df = temp_df.withColumn("class", lit(category))

        if df is None:
            df = temp_df
        else:
            df = df.union(temp_df)

Lets write the dataf-rame to a file in "CSV" format, this will also take quite some time:

In [ ]:
if Path(data_dir + data_csv).exists():
    shutil.rmtree(data_dir + data_csv)

In [ ]:
df = df.repartition(1)

In [ ]:
df = df.drop_duplicates()

In [ ]:
df.write.option("header", "true").csv(data_dir + data_csv)

Now we should have a CSV file with our contents